In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; }</style>")

In [3]:
import new_functions as fn
fn = reload(fn)
import functions as fn2
fn2 = reload(fn2)

In [4]:
import os
import numpy as np
import lsst.daf.persistence as dafPersist
import lsst.afw.display.ds9 as ds9
# import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage

import lsst.afw.geom as afwGeom
import lsst.afw.detection as afwDetect
import pyfits as pf
%matplotlib inline
import scipy
import scipy.ndimage as ndimage
import pylab as plt
import sys

N_AMPS = 16
HISTMIN = -1000
HISTMAX = 10000
NBINS = HISTMAX - HISTMIN + 1

In [ ]:
# files = fn.GetFiles('/Users/mfisherlevine/Desktop/fe55_113-03/','*.fits')
files = fn.GetFiles('/Users/mfisherlevine/Downloads/113-03/fe55/15041/','*fe55_bias*.fits')

print len(files)

In [ ]:
exp_total = 0.
for filenum, filename in enumerate(files):
#     if filenum%1==0: print 'Processing file %s of %s'%(filenum, len(files)); sys.stdout.flush()
    f = pf.open(filename)
    exptime = f[0].header['EXPTIME']
    exp_total += exptime
    print 'Exptime = %s s for %s'%(exptime, filename)
    f.close()
print 'Total exposure time = %s hours'%(exp_total/3600.)

In [ ]:
hists = [np.zeros(NBINS, dtype=np.float64) for _ in range(N_AMPS+1)]
for filenum, filename in enumerate(files):
    if filenum%10==0: print 'Processing file %s of %s'%(filenum, len(files)); sys.stdout.flush()
    f = pf.open(filename)
    for amp in xrange(1,1+N_AMPS):
        n_rows, n_cols = f[amp].data.shape
        data = f[amp].data[:,523:]
#         ds9.mtv(afwImage.ImageF(data))
#         break
        if (amp==5) and (filenum==0):
            print 'sent image'
            ds9.mtv(afwImage.ImageF(f[amp].data[:,10:]))
        oscan = np.median(f[amp].data[:,523:n_cols])
#         data -= oscan
#         fn2.ShowSpot(data[:,1:513], half_stretch=3)
#         break
        hist_vals = ndimage.histogram(data, HISTMIN, HISTMAX, NBINS)
        hists[amp] += hist_vals
    f.close()

In [ ]:
for amp in range(1,N_AMPS+1):
    fig = plt.figure(figsize=(10,4))
    ax = plt.subplot(111)
    xpoints = np.linspace(HISTMIN, HISTMAX, NBINS)
#     plt.semilogy(xpoints, hists[amp])
    plt.step(xpoints, hists[amp])
#     print hists[amp][fn2.indexOfMax(hists[amp])-30:fn2.indexOfMax(hists[amp])+10]
#     break
    ax.semilogy()
#     plt.xlim((-50,50))
#     plt.xlim((-50,1000))
    plt.xlim(fn2.indexOfMax(hists[amp])+HISTMIN - 100, fn2.indexOfMax(hists[amp])+HISTMIN + 30)
    plt.title('Amp %s'%(amp))
    plt.show()

In [ ]:
hists = [np.zeros(NBINS, dtype=np.float64) for _ in range(N_AMPS+1)]

n_sigma_thr = 6.

for filenum, filename in enumerate(files):
    if filenum%10==0: print 'Processing file %s of %s'%(filenum, len(files)); sys.stdout.flush()
    f = pf.open(filename)
    for amp in xrange(1,1+N_AMPS):
        n_rows, n_cols = f[amp].data.shape
        data = f[amp].data[:,1:513]
        oscan = np.median(f[amp].data[:,513:n_cols])
        data -= oscan
        mean, sigma = fn2.getClippedMeanandStddev(data,nsig=3)
        hist_vals = ndimage.histogram(data[data >= mean + (n_sigma_thr*sigma)], HISTMIN, HISTMAX, NBINS)
        hists[amp] += hist_vals
    f.close()

In [ ]:
fig = plt.figure(figsize=(10,4))

for amp in range(1,N_AMPS+1):
    xpoints = np.linspace(HISTMIN, HISTMAX, NBINS)
    plt.semilogy(xpoints, hists[amp])
    plt.xlim((0,600))
#     plt.title('Amp %s'%(amp-1))
#     plt.show()

In [ ]:
# hists = [np.zeros(NBINS, dtype=np.float64) for _ in range(N_AMPS+1)]
# n_sigma_thr = 6.

# THRESHOLD = 40
# N_PIX_MIN = 1
# GROW = 1
# ISOTROPIC = True
# VERBOSE = False

# fluxes = [[] for _ in range(N_AMPS)]

# for filenum, filename in enumerate(files):
#     if filenum%10==0: print 'Processing file %s of %s'%(filenum, len(files)); sys.stdout.flush()
#     f = pf.open(filename)
#     for amp in xrange(1,1+N_AMPS):
#         n_rows, n_cols = f[amp].data.shape
#         data = f[amp].data[:,1:513]
#         oscan = np.median(f[amp].data[:,513:n_cols])
#         data -= oscan
        
        
#         image = afwImage.ImageF(data)
#         maskedImg = afwImage.MaskedImageF(image)
#         exposure = afwImage.ExposureF(maskedImg)

#         threshold = afwDetect.Threshold(THRESHOLD, afwDetect.Threshold.VALUE)
#         footPrintSet = afwDetect.FootprintSet(exposure.getMaskedImage(), threshold, "DETECTED", N_PIX_MIN)
#         footPrintSet = afwDetect.FootprintSet(footPrintSet, GROW, ISOTROPIC)

#         footPrints = footPrintSet.getFootprints()
#         if VERBOSE: print "Found %s footprints in amp %s of file %s"%(footPrints.size(), amp, filename)
# #         total_found += footPrints.size()

# #         if footPrints.size() >= 1000: # files with bright defects cause all sorts of problems
# #             print "Bad file - skipping amp %s in %s"%(amp, filename)
# #             continue

#         for pointnum, footprint in enumerate(footPrints):
#             flux = fn2.SumFootprintPixels(footprint,image)
#             fluxes[amp-1].append(flux)
#     f.close()

In [ ]:
hists = [np.zeros(NBINS, dtype=np.float64) for _ in range(N_AMPS+1)]

n_sigma_thr = 4.
N_PIX_MIN = 2
GROW = 0
ISOTROPIC = True
VERBOSE = False

thrs = []

fluxes = [[] for _ in range(N_AMPS)]

for filenum, filename in enumerate(files):
    if filenum%10==0: print 'Processing file %s of %s'%(filenum, len(files)); sys.stdout.flush()
    f = pf.open(filename)
    for amp in xrange(1,1+N_AMPS):
        n_rows, n_cols = f[amp].data.shape
        data = f[amp].data[:,1:513]
        oscan = np.median(f[amp].data[:,513:n_cols])
        data -= oscan
        
        image = afwImage.ImageF(data)
        maskedImg = afwImage.MaskedImageF(image)
        exposure = afwImage.ExposureF(maskedImg)

        mean, sigma = fn2.getClippedMeanandStddev(data, nsig=5)
        THRESHOLD = mean + (n_sigma_thr*sigma)
#         print 'Threshold = %s'%THRESHOLD
        thrs.append(THRESHOLD)
        
        threshold = afwDetect.Threshold(THRESHOLD, afwDetect.Threshold.VALUE)
        footPrintSet = afwDetect.FootprintSet(exposure.getMaskedImage(), threshold, "DETECTED", N_PIX_MIN)
        footPrintSet = afwDetect.FootprintSet(footPrintSet, GROW, ISOTROPIC)

        footPrints = footPrintSet.getFootprints()
        if VERBOSE: print "Found %s footprints in amp %s of file %s"%(footPrints.size(), amp, filename)
#         total_found += footPrints.size()

#         if footPrints.size() >= 1000: # files with bright defects cause all sorts of problems
#             print "Bad file - skipping amp %s in %s"%(amp, filename)
#             continue

        for pointnum, footprint in enumerate(footPrints):
            flux = fn2.SumFootprintPixels(footprint,image)
            fluxes[amp-1].append(flux)
    f.close()

In [ ]:
from scipy.signal import argrelmax 
from scipy.signal import find_peaks_cwt

In [ ]:
def ThresholdList(input_list, threshold, default_value=0, return_as_array=False):
    if return_as_array: return np.asarray([val if val>=threshold else default_value for val in input_list])
    return [val if val>=threshold else default_value for val in input_list]

In [ ]:
HISTMIN = 200.
HISTMAX = 800.
NBINS = HISTMAX - HISTMIN + 1

xpoints = np.linspace(HISTMIN, HISTMAX, NBINS)
print 'Binsize = %s'%(xpoints[1]-xpoints[0])

for amp in range(N_AMPS):
    fig = plt.figure(figsize=(12,4))
    
    histvals = ndimage.histogram(np.asarray(fluxes[amp]),HISTMIN, HISTMAX, NBINS)
#     peaks = find_peaks_cwt(histvals, widths=np.array([10]))#, wavelet=scipy.signal.wavelets.morlet)
#     peaks = argrelmax(ThresholdList(hist_vals, 0.1*max(hist_vals),return_as_array=True), order=20)
    peaks = argrelmax(np.asarray([val if val>=0.1*max(hist_vals) else 0 for val in histvals]), order=20)
#     for a,b in zip(his\t_vals, ThresholdList(hist_vals, 0.1*max(hist_vals),return_as_array=True)):
#         print a,b
    
    print peaks
    
    plt.step(xpoints, histvals)
    plt.plot([peak + HISTMIN for peak in peaks], [2000. for _ in range(len(peaks))], 'o')
    #     plt.semilogy(xpoints, histvals)
#     plt.xlim((-200,12000))
    plt.title('THR = %s, GROW = %s'%(THRESHOLD, GROW))
    plt.show()


In [ ]:
def FitGaussian(binned_data, std_dev, initial_pars=None, return_fitted_points=False, n_fitted_points=1000, print_values=False, plot=False):
    '''Data should be histogrammed, with the form data[hist_vals,]
    initial_pars takes the form [A, mu, sigma]'''
    from scipy.optimize import curve_fit
    def gauss(x, *p):
        A, mu, sigma = p
        return A*np.exp(-(x-mu)**2/(2.*sigma**2))
    
    bin_centres = binned_data[:,0]
    bin_values  = binned_data[:,1]

    if initial_pars is None:
        max_bin_value = max(binned_data[:,1])
        max_bin = np.where(binned_data[:,1]==max_bin_value)[0][0]
        max_bin_center = binned_data[max_bin,0]
        a0     = max_bin_value
        mu0    = max_bin_center
        sigma0 = std_dev
        initial_pars = [a0, mu0, sigma0]

    if print_values: print 'Initial parameters = %.2f, %.2f, %.2f'%(initial_pars[0],initial_pars[1],initial_pars[2])
    coeff, var_matrix = curve_fit(gauss, bin_centres, bin_values, p0=initial_pars, sigma=[np.sqrt(_) if _>=1 else 1 for _ in bin_values])
    if print_values: print 'Fitted parameters  = %.2f, %.2f, %.2f'%(coeff[0],coeff[1],coeff[2])
    
    if print_values:
        print 'Mean  = %.5f +/- %.6f'%(coeff[1], var_matrix[1,1])
        print 'Sigma = %.5f +/- %.6f'%(coeff[2], var_matrix[2,2])

    xpoints = np.linspace(min(bin_centres), max(bin_centres),n_fitted_points)
    fitted_points = gauss(xpoints, *coeff)
    
    if plot:
        plt.plot(binned_data[:,0], binned_data[:,1], 'x')
        plt.plot(xpoints, fitted_points, label='Fit')
        plt.show()
    
    if return_fitted_points:
        return xpoints, fitted_points, coeff
    return coeff

def FitDoubleGaussian(binned_data, std_dev, second_gaus_offset=10, initial_pars=None, return_fitted_points=False, n_fitted_points=1000, print_values=False, plot=False):
    '''Data should be histogrammed, with the form data[hist_vals,]
    initial_pars takes the form [A, mu, sigma]'''
    from scipy.optimize import curve_fit
    def double_gauss(x, *p):
        A, mu, sigma, A2, mu2, sigma2 = p
        return A*np.exp(-(x-mu)**2/(2.*sigma**2)) + A2*np.exp(-(x-mu2)**2/(2.*sigma2**2))
    
    bin_centres = binned_data[:,0]
    bin_values  = binned_data[:,1]

    if initial_pars is None:
        max_bin_value = max(binned_data[:,1])
        max_bin = np.where(binned_data[:,1]==max_bin_value)[0][0]
        max_bin_center = binned_data[max_bin,0]
        a0     = max_bin_value
        mu0    = max_bin_center
        sigma0 = std_dev
        a20    = max_bin_value * 0.12
        mu20   = mu0 + second_gaus_offset
        sigma20 = sigma0
        initial_pars = [a0, mu0, sigma0, a20, mu20, sigma20]

    if print_values: print 'Initial parameters = %.2f, %.2f, %.2f, %.2f, %.2f, %.2f'%(initial_pars[0],initial_pars[1],initial_pars[2],initial_pars[3],initial_pars[4],initial_pars[5])
    coeff, var_matrix = curve_fit(double_gauss, bin_centres, bin_values, p0=initial_pars, sigma=[np.sqrt(_) if _>=1 else 1 for _ in bin_values])
    if print_values: print 'Fitted parameters  = %.2f, %.2f, %.2f, %.2f, %.2f, %.2f'%(coeff[0],coeff[1],coeff[2],coeff[3],coeff[4],coeff[5])
    
    if print_values:
        print 'Mean  = %.5f +/- %.6f'%(coeff[1], var_matrix[1,1])
        print 'Sigma = %.5f +/- %.6f'%(coeff[2], var_matrix[2,2])

    xpoints = np.linspace(bin_centres[0], bin_centres[-1],n_fitted_points)
    fitted_points = double_gauss(xpoints, *coeff)
    
    if plot:
        plt.plot(binned_data[:,0], binned_data[:,1], 'x')
        plt.plot(xpoints, fitted_points, '-r', label='Fit')
        plt.show()
    
    if return_fitted_points:
        return xpoints, fitted_points, coeff
    return coeff

    

In [ ]:
HISTMIN = 200.
HISTMAX = 800.
NBINS = HISTMAX - HISTMIN + 1

xpoints = np.linspace(HISTMIN, HISTMAX, NBINS)
print 'Binsize = %s'%(xpoints[1]-xpoints[0])

k_alpha_peaks = []
k_beta_peaks = []

for amp in range(N_AMPS):
    fig = plt.figure(figsize=(12,4))
    ax = plt.subplot(111)
    
    histvals = ndimage.histogram(np.asarray(fluxes[amp]),HISTMIN, HISTMAX, NBINS)
#     peaks = argrelmax(np.asarray([val if val>=0.1*max(hist_vals) else 0 for val in histvals]), order=20)
    bin_centers = xpoints + ((xpoints[1]-xpoints[0])/2.)
    
    fit_data = np.ndarray((len(histvals),2))
    fit_data[:,1] = histvals
    fit_data[:,0] = bin_centers
    
    xpoints_fit, fitted_points, fit_pars = FitDoubleGaussian(fit_data, 8., second_gaus_offset=50, print_values=True, plot=False, return_fitted_points=True)
#     xpoints_fit, fitted_points, fit_pars = FitGaussian(fit_data, 8., print_values=True, plot=False, return_fitted_points=True)
    k_alpha_peaks.append(fit_pars[1])
    k_beta_peaks.append(fit_pars[4])

    plt.step(xpoints, histvals)
    plt.plot(xpoints_fit-1, fitted_points, '-r')

    ax.set_yscale("log", nonposy='clip')


#     plt.plot([peak + HISTMIN for peak in peaks], [2000. for _ in range(len(peaks))], 'o')
    plt.xlim((450,750))
    plt.ylim((1,max(histvals)+500))

    plt.title('Amp %s, THR = %s, GROW = %s'%(amp, THRESHOLD, GROW))
    plt.show()
#     break


In [ ]:
K_ALPHA_1   = 5898.75
K_ALPHA_2   = 5887.65
K_ALPHA_NOM = ((1.97561*K_ALPHA_1) + (K_ALPHA_2)) / (2.97561) # weighting by the ratio of the two lines
K_BETA      = 6490.45
DELTA_E     = K_ALPHA_NOM - K_BETA

In [ ]:
temp_in_kelvin = 173.
PAIR_ENERGY = 3.73 - (0.000131*temp_in_kelvin)
electrons_per_K_ALPHA = K_ALPHA_NOM/PAIR_ENERGY
electrons_per_E_DELTA = (DELTA_E/PAIR_ENERGY)
print 'Pair creation energy = %s'%PAIR_ENERGY
print '# e- per K_alpha hit = %s'%(electrons_per_K_ALPHA)

In [ ]:
gains = []
for amp in range(N_AMPS):
    gain = electrons_per_K_ALPHA/k_alpha_peaks[amp]
    gains.append(gain)
    print 'Amp %2i gain = %.3f'%(amp, gain)

print '\n*********\n'

gains_delta_e = []
for amp in range(N_AMPS):
    gain = electrons_per_E_DELTA/(k_alpha_peaks[amp] - k_beta_peaks[amp])
    gains_delta_e.append(gain)
    print 'Amp %2i gain = %.3f'%(amp, gain)

In [ ]:
plt.plot(gains, 'o')
plt.plot(gains_delta_e, 'o')
plt.show()
plt.plot(gains, gains_delta_e, 'x')

In [ ]:
BNL_amp_gains = [2.83, 2.79, 2.87, 2.81, 2.83,2.82, 2.83, 2.96, 2.75, 2.70, 2.72, 2.69, 2.74, 2.74, 2.69, 2.74]
plt.plot(BNL_amp_gains, 'o')
plt.show()
ys, xs, patches = plt.hist(BNL_amp_gains, bins=5)

In [ ]:
plt.plot(BNL_amp_gains, gains, 'o')